# 환경 설정

In [3]:
!pip install numpy==1.23.1
# !pip install numpy==1.22.1
# !pip install numpy==1.19.5

In [ ]:
'''
import os
import random
from copy import deepcopy
import numpy as np
import tables
import json
import itertools
from tqdm import tqdm
import torch
import torch.utils.data as data
import logging
from transformers import BertTokenizer, BertForMaskedLM, AdamW
import math
from torch.nn import CrossEntropyLoss

logger = logging.getLogger(__name__)

class DialogTransformerDataset(data.Dataset):
    # ... (unchanged dataset class code) ...

class HBertMseEuopDataset(DialogTransformerDataset):
    # ... (unchanged dataset class code) ...

def load_dict(filename):
    return json.loads(open(filename, "r").readline())

def load_vecs(fin):
    """read vectors (2D numpy array) from a hdf5 file"""
    h5f = tables.open_file(fin)
    h5vecs= h5f.root.vecs
    vecs=np.zeros(shape=h5vecs.shape,dtype=h5vecs.dtype)
    vecs[:]=h5vecs[:]
    h5f.close()
    return vecs

def save_vecs(vecs, fout):
    fvec = tables.open_file(fout, 'w')
    atom = tables.Atom.from_dtype(vecs.dtype)
    filters = tables.Filters(complib='blosc', complevel=5)
    ds = fvec.create_carray(fvec.root,'vecs', atom, vecs.shape,filters=filters)
    ds[:] = vecs
    print('done')
    fvec.close()

if __name__ == '__main__':
    # File and tokenizer setup
    input_dir = './data/reddit/'
    VALID_FILE = input_dir + 'train.h5'
    task = 'test_ctx'  # or 'test_utt'

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    if task == 'test_utt':
        dataset = DialogTransformerDataset(VALID_FILE, tokenizer, utt_masklm=True, utt_sop=True)
    elif task == 'test_ctx':
        dataset = DialogTransformerDataset(VALID_FILE, tokenizer, context_shuf=True, context_masklm=False)
    else:
        dataset = DialogTransformerDataset(VALID_FILE, tokenizer)

    train_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=1)
    eval_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=32, shuffle=False, num_workers=1)

    # Model setup
    model = BertForMaskedLM.from_pretrained('bert-base-uncased')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Optimizer and loss function
    learning_rate = 5e-5
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    criterion = CrossEntropyLoss()

    # Training settings
    epochs = 3
    history = {'train_loss': [], 'eval_loss': [], 'train_accuracy': [], 'eval_accuracy': [], 'train_perplexity': [], 'eval_perplexity': []}
    all_predictions = []
    all_targets = []

    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            optimizer.zero_grad()
            context, context_utts_attn_mask, context_attn_mask, context_mlm_target, context_position_perm_id, context_position_ids, response = batch

            input_ids = context.view(-1, context.size(-1)).to(device)
            attention_mask = context_attn_mask.view(-1, context_attn_mask.size(-1)).to(device)
            labels = context_mlm_target.view(-1, context_mlm_target.size(-1)).to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = torch.argmax(logits, dim=-1)
            correct_predictions += (preds == labels).sum().item()
            total_predictions += labels.numel()

        avg_train_loss = running_loss / len(train_loader)
        train_accuracy = correct_predictions / total_predictions
        train_perplexity = math.exp(avg_train_loss)

        history['train_loss'].append(avg_train_loss)
        history['train_accuracy'].append(train_accuracy)
        history['train_perplexity'].append(train_perplexity)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Train Perplexity: {train_perplexity:.4f}")

        # Evaluation loop
        model.eval()
        eval_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        eval_predictions = []
        eval_targets = []

        with torch.no_grad():
            for batch in tqdm(eval_loader, desc=f"Evaluation Epoch {epoch+1}"):
                context, context_utts_attn_mask, context_attn_mask, context_mlm_target, context_position_perm_id, context_position_ids, response = batch

                input_ids = context.view(-1, context.size(-1)).to(device)
                attention_mask = context_attn_mask.view(-1, context_attn_mask.size(-1)).to(device)
                labels = context_mlm_target.view(-1, context_mlm_target.size(-1)).to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                logits = outputs.logits

                eval_loss += loss.item()
                preds = torch.argmax(logits, dim=-1)
                correct_predictions += (preds == labels).sum().item()
                total_predictions += labels.numel()

                eval_predictions.extend(preds.cpu().numpy())
                eval_targets.extend(labels.cpu().numpy())

        avg_eval_loss = eval_loss / len(eval_loader)
        eval_accuracy = correct_predictions / total_predictions
        eval_perplexity = math.exp(avg_eval_loss)

        history['eval_loss'].append(avg_eval_loss)
        history['eval_accuracy'].append(eval_accuracy)
        history['eval_perplexity'].append(eval_perplexity)

        print(f"Epoch {epoch+1}/{epochs}, Eval Loss: {avg_eval_loss:.4f}, Eval Accuracy: {eval_accuracy:.4f}, Eval Perplexity: {eval_perplexity:.4f}")

        all_predictions.extend(eval_predictions)
        all_targets.extend(eval_targets)

    print("Training complete.")
'''

In [1]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install sentencepiece
!pip install transformers
!pip install torch>=1.8.1
!pip install tqdm
!pip install torchsummary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 11.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292696 sha256=6f3b683d028de7186cc43d46db8137633af10a864f87b8a510ad379754d91fe0
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp


In [2]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-k48gm03h/kobert-tokenizer_5569dc6d20294700a7056b95706fb599
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-k48gm03h/kobert-tokenizer_5569dc6d20294700a7056b95706fb599
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=4aaae081534fd244774d8f8af1c10cdae38b19a86a9bd0b93f10a99cccb212bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-jppffb5r/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [4]:
!pip install torchviz

  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4132 sha256=45d1fbcd1dcdb5b43197082f0c17c7503b45e2c7432b1564b0c0b4369015be69
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz


In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## import

In [6]:
import torch
from torch.optim import AdamW
from torch.nn import MSELoss, CrossEntropyLoss
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from torchsummary import summary
from torchviz import make_dot

from transformers import BertModel, BertTokenizer, BertForMaskedLM, BertConfig, get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from tqdm import tqdm

import pandas as pd
import numpy as np

import random
import math
import os

import seaborn as sns
import matplotlib.pyplot as plt

# 토크나이저

In [54]:
# KoBERT 토크나이저 불러오기
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


# 데이터셋 & 데이터로더 정의

# 전처리된 데이터셋 파일 불러오기

In [8]:
folder_path = '/content/drive/MyDrive/NLP/Dataset/240527'
file_name = '[haein]dataset_cleansing+koSpacing+drop_duplicate_240527.csv'

df = pd.read_csv(folder_path+'/'+file_name)
df

,id,sort,NO1,column,내담자,상담자,token_len,corrected_column
0,2000001.0,1,0.0,"네, 안녕하세요. 요즘 정말 좋은 일이 많이 있었거든요.",1.0,0.0,16,"네, 안녕하세요.요즘 정말 좋은 일이 많이 있었거든요."
1,2000001.0,2,0.0,"네, 맞아요. 정말 행복한 시기를 보내고 있어요.",1.0,0.0,15,"네, 맞아요.정말 행복한 시기를 보내고 있어요."
2,2000001.0,3,0.0,그렇죠. 어려운 시기를 견뎌내고 이렇게 행복한 날들을 맞이할 수 있어 감사해요.,1.0,0.0,25,그렇죠. 어려운 시기를 견뎌내고 이렇게 행복한 날들을 맞이할 수 있어 감사해요.
3,2000001.0,4,0.0,맞아요. 정말 기분 좋은 일이에요.,1.0,0.0,10,맞아요.정말 기분 좋은 일이에요.
4,2000001.0,5,0.0,"네, 감사합니다. 상담해주셔서 정말 고맙습니다.",1.0,0.0,15,"네, 감사합니다. 상담해주셔서 정말 고맙습니다."
...,...,...,...,...,...,...,...,...
48049,8727.0,15,0.0,"그리고 저희는 더 자세하게 이야기를 나누어봐야겠지만, 가족들이 함께 겪는 이런 힘...",0.0,1.0,34,"그리고 저희는 더 자세하게 이야기를 나누어봐야겠지만, 가족들이 함께 겪는 이런 힘든..."
48050,8727.0,16,0.0,아.. 그렇게 이해해야겠어요. 고맙습니다.,1.0,0.0,13,아.. 그렇게 이해해야겠어요. 고맙습니다.
48051,8727.0,17,0.0,"더욱 구체적인 이야기를 나누어봐야겠지만, 지금처럼 서로 이해하고, 공감해주는 마음...",0.0,1.0,38,"더욱 구체적인 이야기를 나누어봐야겠지만, 지금처럼 서로 이해하고, 공감해주는 마음을..."
48052,8727.0,18,0.0,"네, 이번 상담으로 좀 더 지혜롭게 가족과 지낼 수 있는 방법을 찾을 수 있을 것...",1.0,0.0,25,"네, 이번 상담으로 좀 더 지혜롭게 가족과 지낼 수 있는 방법을 찾을 수 있을 것 ..."


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(max_features=1000, min_df=5)  # 상위 1000개의 단어로 제한
tfidf_vectorizer.fit(df['column'])  # 전체 데이터의 텍스트를 사용하여 벡터라이저를 학습시킴

# 상위 1000개의 단어 출력
top_1000_words = tfidf_vectorizer.get_feature_names_out()
print("TF-IDF 상위 1000개 단어:", top_1000_words)

# 원본 텍스트에서 TF-IDF 상위 1000개 단어만 남기기
def filter_text_by_top_words(text, top_words):
    filtered_words = [word for word in text.split() if word in top_words]
    return ' '.join(filtered_words)

df['filtered_text'] = df['column'].apply(lambda x: filter_text_by_top_words(x, top_1000_words))

TF-IDF 상위 1000개 단어: ['가끔은' '가는' '가능성이' '가면' '가장' '가족' '가족과' '가족들과' '가족들이' '가지' '가지고' '가지는'
 '가진' '가질' '가치를' '간단한' '갈등이' '감사합니다' '감사해요' '감정에' '감정을' '감정이' '갑자기' '갖고'
 '갖는' '같고' '같네요' '같다는' '같습니다' '같아' '같아서' '같아서요' '같아요' '같은' '같은데' '같이' '거라'
 '거라고' '거에요' '거예요' '거의' '거죠' '걱정을' '걱정이' '건가요' '건강한' '건지' '걸까요' '것도' '것들을'
 '것들이' '것만으로도' '것보다' '것부터' '것에' '것으로' '것은' '것을' '것이' '것인지' '것입니다' '것처럼'
 '겪고' '결과가' '결과를' '결국' '결혼을' '경우' '경우가' '경우도' '경우에는' '경험을' '경험이' '계속'
 '계속해서' '계시군요' '계시나요' '계시는' '계신' '계신가요' '계획을' '고마워요' '고맙습니다' '고민으로' '고민을'
 '고민이' '고민하고' '고통을' '공부를' '과정에서' '관계가' '관계를' '관련된' '관심을' '괜찮은' '굉장히'
 '구체적으로' '궁금한' '규칙적인' '균형을' '그건' '그걸' '그것도' '그것은' '그것을' '그것이' '그게' '그냥'
 '그동안' '그래도' '그래서' '그래요' '그러게요' '그러나' '그러니까' '그러면' '그러면서' '그러셨군요' '그러시군요'
 '그러한' '그런' '그런데' '그럴' '그럼' '그렇게' '그렇군요' '그렇다면' '그렇습니다' '그렇지' '그리고' '그만큼'
 '그에' '그저' '극복할' '근데' '긍정적으로' '긍정적인' '기분을' '기분이' '기회가' '기회를' '깊이' '꾸준히'
 '나가는' '나갈' '나누고' '나누는' '나누면서' '나누어' '나눌' '나는' '나를' '나쁜' '나서' '나아갈' '나아질'
 '나은' '나의' '내가' '내에서' '내용을' '내용이' '

In [11]:
# 데이터에 NaN 값이 있는지 확인
print(df.isnull().sum())

id                  0
sort                0
NO1                 0
column              0
내담자                 0
상담자                 0
token_len           0
corrected_column    0
filtered_text       0
dtype: int64


In [12]:
# 데이터를 학습용과 평가용으로 분리
train_df, eval_df = train_test_split(df, test_size=0.1, random_state=42)

# 데이터프레임 확인
print(train_df[['column', 'filtered_text']].head())

                                                  column  \
34920                          그렇다면 어떤 점이 가장 힘들게 느껴지시나요?   
1661    지금 바로 희망이 보이지 않는 건 당연해요. 그래도 조금씩, 천천히 나아가다 보면...   
10894   우선, 대인관계에서 불안감을 느끼시면 그걸 이해하고 수용할 수 있는 관점을 갖는 ...   
29042   잘 이해합니다. 기분 나쁜 일을 당하셨군요. 혹시 인간관계가 문제가 된 이유를 파...   
3734    그럴 때마다 정말 힘드시겠어요. 스트레스가 많아서 더 힘들어지는 것 같네요. 어떤...   

                                           filtered_text  
34920                                  그렇다면 어떤 점이 가장 힘들게  
1661   지금 바로 않는 그래도 천천히 보면 어느 삶의 의미를 당신은 정말 소중한 잊지 주변...  
10894  불안감을 그걸 이해하고 있는 갖는 것이 상대방의 상대방과 자신의 의견을 표현하는 것...  
29042                                   나쁜 일을 혹시 문제가 이유를  
3734   그럴 때마다 정말 스트레스가 많아서 어떤 것들이 스트레스를 그리고 어떤 상황에서 가...  


In [13]:
print(train_df.shape, eval_df.shape)

(43248, 9) (4806, 9)


In [14]:
print(train_df.head())

              id  sort  NO1  \
34920     5583.0     9  0.0   
1661   1001006.0    11  0.0   
10894      500.0    15  0.0   
29042     4297.0     3  0.0   
3734   3000082.0     9  0.0   

                                                  column  내담자  상담자  token_len  \
34920                          그렇다면 어떤 점이 가장 힘들게 느껴지시나요?  0.0  1.0         14   
1661    지금 바로 희망이 보이지 않는 건 당연해요. 그래도 조금씩, 천천히 나아가다 보면...  0.0  1.0         93   
10894   우선, 대인관계에서 불안감을 느끼시면 그걸 이해하고 수용할 수 있는 관점을 갖는 ...  0.0  1.0         87   
29042   잘 이해합니다. 기분 나쁜 일을 당하셨군요. 혹시 인간관계가 문제가 된 이유를 파...  0.0  1.0         28   
3734    그럴 때마다 정말 힘드시겠어요. 스트레스가 많아서 더 힘들어지는 것 같네요. 어떤...  0.0  1.0         46   

                                        corrected_column  \
34920                         그렇다면 어떤 점이 가장 힘들게 느껴 지시나요?   
1661   지금 바로 희망이 보이지 않는 건 당연해요. 그래도 조금씩, 천천히 나아가다 보면 ...   
10894  우선, 대인관계에서 불안감을 느끼시면 그걸 이해하고 수용할 수 있는 관점을 갖는 것...   
29042  잘 이해합니다.기분 나쁜 일을 당하셨군요. 혹시 인간관계가 문제가 된 이유를 파악하...   
3734   그럴 때마다 정말 힘드시겠어요. 스

In [19]:
import os
import random
from copy import deepcopy
import numpy as np
import tables
import json
import itertools
from tqdm import tqdm
import torch
import torch.utils.data as data
import logging
from transformers import BertTokenizer, BertForMaskedLM, AdamW
import math
from torch.nn import CrossEntropyLoss

logger = logging.getLogger(__name__)

class DialogTransformerDataset(data.Dataset):
    """
    A base class for Transformer dataset
    """
    def __init__(self, file_path, tokenizer,
                 min_num_utts=1, max_num_utts=7, max_utt_len=30,
                 block_size=256, utt_masklm=False, utt_sop=False,
                 context_shuf=False, context_masklm=False):
        # 1. Initialize file path or list of file names.
        """read training sentences(list of int array) from a hdf5 file"""
        self.tokenizer = tokenizer
        self.min_num_utts = min_num_utts #if not context_shuf and not context_masklm else 3
        self.max_num_utts = max_num_utts
        self.max_utt_len =max_utt_len
        self.block_size = block_size # segment size to train BERT. when set -1 by default, use indivicual sentences(responses) as BERT inputs.
                            # Otherwise, clip a block from the context.

        self.utt_masklm = utt_masklm
        self.utt_sop =utt_sop
        self.context_shuf =context_shuf
        self.context_masklm =context_masklm

        self.rand_utt = [tokenizer.mask_token_id]*(max_utt_len-1) + [tokenizer.sep_token_id] # update during loading

        # a cache to store context and response that are longer than min_num_utts
        self.cache = [[tokenizer.mask_token_id]*max_utt_len]*max_num_utts, [tokenizer.mask_token_id]*max_utt_len

        self.perm_list = [list(itertools.permutations(range(L))) for L in range(1, max_num_utts+1)]
        print("loading data...")
        table = tables.open_file(file_path)
        self.contexts = table.get_node('/column')[:].astype(np.long)
        #self.knowlege = table.get_node('/knowledge')[:].astype(np.long)
        self.index = table.get_node('/indices')[:]
        self.data_len = self.index.shape[0]
        print("{} entries".format(self.data_len))

    def __getitem__(self, offset):
        index = self.index[offset]
        pos_utt, ctx_len, res_len,  = index['pos_utt'], index['ctx_len'], index['res_len']
        #pos_knowl, knowl_len = index['pos_knowl'], index['knowl_len']

        ctx_len = min(ctx_len, self.block_size) if self.block_size>-1 else ctx_len# trunck too long context

        ctx_arr=self.contexts[pos_utt-ctx_len:pos_utt].tolist()
        res_arr=self.contexts[pos_utt:pos_utt+res_len].tolist()
        #knowl_arr = self.knowledge[pos_knowl:pos_knowl+knowl_len].tolist()

        ## split context array into utterances
        context = []
        tmp_utt = []
        for i, tok in enumerate(ctx_arr):
            tmp_utt.append(ctx_arr[i])
            if tok == self.tokenizer.sep_token_id:
                floor = tmp_utt[0]
                tmp_utt = tmp_utt[1:]
                utt_len = min(len(tmp_utt), self.max_utt_len) # floor is not counted in the utt length
                utt = tmp_utt[:utt_len]
                context.append(utt)  # append utt to context
                tmp_utt=[]  # reset tmp utt
        response = res_arr[1:] # ignore cls token at the begining
        res_len = min(len(response),self.max_utt_len)
        response = response[:res_len-1] + [self.tokenizer.sep_token_id]

        '''
        knowledge = knowl_arr[:]
        knowl_len = min(len(knowledge),self.max_utt_len)
        knowledge = knowledge[:knowl_len-1] + [self.tokenizer.sep_token_id]
        '''

        # balancing by removing short contexts
 #       if len(context)< self.min_num_utts:
 #           context, response = self.cache
 #       else:
 #           self.cache = deepcopy(context), deepcopy(response)
        # end balancing

        num_utts = min(len(context), self.max_num_utts)
        context = context[-num_utts:]

        return context, response #, knowlege

    def list2array(self, L, d1_len, d2_len=0, d3_len=0, dtype=np.long, pad_idx=0):
        '''  convert a list to an array or matrix  '''
        def list_dim(a):
            if type(a)!=list: return 0
            elif len(a)==0: return 1
            else: return list_dim(a[0])+1

        if type(L) is not list:
            print("requires a (nested) list as input")
            return None

        if list_dim(L)==0: return L
        elif list_dim(L) == 1:
            arr = np.zeros(d1_len, dtype=dtype)+pad_idx
            for i, v in enumerate(L): arr[i] = v
            return arr
        elif list_dim(L) == 2:
            arr = np.zeros((d2_len, d1_len), dtype=dtype)+pad_idx
            for i, row in enumerate(L):
                for j, v in enumerate(row):
                    arr[i][j] = v
            return arr
        elif list_dim(L) == 3:
            arr = np.zeros((d3_len, d2_len, d1_len), dtype=dtype)+pad_idx
            for k, group in enumerate(L):
                for i, row in enumerate(group):
                    for j, v in enumerate(row):
                        arr[k][i][j] = v
            return arr
        else:
            print('error: the list to be converted cannot have a dimenson exceeding 3')

    def mask_words(self, utt):
        output_label = []
        tokens = [tok for tok in utt]
        for i, token in enumerate(utt):
            prob = random.random()
            if prob < 0.15 and not token in [self.tokenizer.pad_token_id, self.tokenizer.sep_token_id, self.tokenizer.cls_token_id]:
                prob /= 0.15
                if prob < 0.8:
                    tokens[i] = self.tokenizer.mask_token_id   # 80% randomly change token to mask token
                elif prob < 0.9:
                    tokens[i] = random.randint(5, len(self.tokenizer)-5)# 10% randomly change token to random token
                output_label.append(token)
            else:
                output_label.append(-100)
        return tokens, output_label


    def swap_utt(self, utt):
        utt_sop_label = 0 if random.random()>0.6 or len(utt)<5 else 1
        tokens = [tok for tok in utt]
        utt_len = len(tokens)
        if utt_len == self.max_utt_len: # if utt has reached the maximum length, then remove the last token because we will add a new sep token
            tokens = tokens[:-2]+ [self.tokenizer.sep_token_id]
            utt_len-=1
        sep_pos = random.randrange(2, utt_len-1) # seperate position where tokens to the right are random or coherent contexts

        # new utt
        L_utt, R_utt = tokens[1:sep_pos]+[self.tokenizer.sep_token_id], tokens[sep_pos:]
        swaped_utt = L_utt + R_utt if utt_sop_label ==0 else R_utt + L_utt
        swaped_utt = [self.tokenizer.cls_token_id] + swaped_utt
        utt_attn_mask = [1]*len(swaped_utt)
        # segment_ids
        utt_segment_ids = [0]*(sep_pos+1)+[1]*(utt_len-sep_pos) if utt_sop_label == 0 else [0]*(utt_len-sep_pos+1)+[1]*(sep_pos)

        return swaped_utt, utt_attn_mask, utt_segment_ids, utt_sop_label

    def mask_context(self, context):
        def is_special_utt(utt):
            return len(utt)==3 and utt[1] in [self.tokenizer.mask_token_id, self.tokenizer.sep_token_id, self.tokenizer.cls_token_id]

        utts = [utt for utt in context]
        lm_label = [[-100]*len(utt) for utt in context]
        context_len = len(context)
        assert context_len>1, 'a context to be masked should have at least 2 utterances'

        mlm_probs = [0.0, 0.1, 0.4, 0.7, 0.8, 0.9, 1.0, 1.0, 1.0, 1.0]
        mlm_prob = mlm_probs[context_len-1]

        prob = random.random()
        if prob < mlm_prob:
            i = random.randrange(context_len)
            while is_special_utt(utts[i]):
                i = random.randrange(context_len)
            utt = utts[i]
            prob = prob/mlm_prob
            if prob < 0.8: # 80% randomly change utt to mask utt
                utts[i] = [self.tokenizer.cls_token_id, self.tokenizer.mask_token_id, self.tokenizer.sep_token_id]
            elif prob < 0.9: # 10% randomly change utt to a random utt
                utts[i] = deepcopy(self.rand_utt)
            lm_label[i]= deepcopy(utt)
            #assert len(utts[i]) == len(lm_label[i]), "the size of the lm label is different to that of the masked utterance"
            self.rand_utt = deepcopy(utt) # update random utt
        return utts, lm_label

    def shuf_ctx(self, context):
        perm_label = 0
        num_utts = len(context)
        if num_utts==1:
            return context, perm_label, [0]
        for i in range(num_utts-1): perm_label += len(self.perm_list[i])
        perm_id = int(random.random()*len(self.perm_list[num_utts-1]))
        perm_label += perm_id
        ctx_position_ids = self.perm_list[num_utts-1][perm_id]
        # new context
        shuf_context = [context[i] for i in ctx_position_ids]
        return shuf_context, perm_label, ctx_position_ids

    def __len__(self):
        return self.data_len

<ipython-input-19-63744af71d14>:99: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def list2array(self, L, d1_len, d2_len=0, d3_len=0, dtype=np.long, pad_idx=0):


In [30]:
# KoBERT 토크나이저 불러오기
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [96]:
#DialogBert masking
'''
import pandas as pd
import torch
import math
from transformers import BertTokenizer, BertForMaskedLM, AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

def load_dict(filename):
    import json
    with open(filename, "r") as f:
        return json.loads(f.readline())

def load_vecs(fin):
    import tables
    import numpy as np
    """read vectors (2D numpy array) from a hdf5 file"""
    h5f = tables.open_file(fin)
    h5vecs = h5f.root.vecs
    vecs = np.zeros(shape=h5vecs.shape, dtype=h5vecs.dtype)
    vecs[:] = h5vecs[:]
    h5f.close()
    return vecs

def save_vecs(vecs, fout):
    import tables
    fvec = tables.open_file(fout, 'w')
    atom = tables.Atom.from_dtype(vecs.dtype)
    filters = tables.Filters(complib='blosc', complevel=5)
    ds = fvec.create_carray(fvec.root, 'vecs', atom, vecs.shape, filters=filters)
    ds[:] = vecs
    print('done')
    fvec.close()

class DialogTransformerDataset(torch.utils.data.Dataset):
    def __init__(self, file_path, tokenizer, min_num_utts=2, max_num_utts=15, max_utt_len=50, block_size=512,
                 utt_masklm=False, utt_sop=False, context_shuf=False, context_masklm=False):
        self.tokenizer = tokenizer
        self.min_num_utts = min_num_utts
        self.max_num_utts = max_num_utts
        self.max_utt_len = max_utt_len
        self.block_size = block_size
        self.utt_masklm = utt_masklm
        self.utt_sop = utt_sop
        self.context_shuf = context_shuf
        self.context_masklm = context_masklm

        print("Loading data...")
        df = pd.read_csv(file_path)
        self.contexts = df['corrected_column'].values  # Adjust according to your CSV column names
        # self.knowledge = df['knowledge'].values  # Uncomment if you have this column

    def __len__(self):
        return len(self.contexts)

    def __getitem__(self, idx):
        context = self.contexts[idx]
        encoded_dict = self.tokenizer.encode_plus(
            context,
            add_special_tokens=True,
            max_length=self.max_utt_len,
            pad_to_max_length=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_dict['input_ids'].squeeze()
        attention_mask = encoded_dict['attention_mask'].squeeze()

        if self.utt_masklm or self.context_masklm:
            input_ids, labels = self.mask_tokens(input_ids)
        else:
            labels = input_ids

        return input_ids, attention_mask, labels, torch.tensor([]), torch.tensor([]), torch.tensor([]), torch.tensor([])


    def __getitem__(self, idx):
        context = self.contexts[idx]
        encoded_dict = self.tokenizer.encode_plus(
            context,
            add_special_tokens=True,
            max_length=self.max_utt_len,
            pad_to_max_length=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_dict['input_ids'].squeeze()
        attention_mask = encoded_dict['attention_mask'].squeeze()

        # You need to implement other outputs as required
        return input_ids, attention_mask, torch.tensor([]), torch.tensor([]), torch.tensor([]), torch.tensor([]), torch.tensor([])

    def mask_tokens(self, input_ids):
        labels = input_ids.clone()
        probability_matrix = torch.full(labels.shape, 0.15)
        special_tokens_mask = [
            self.tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
        ]
        probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
        masked_indices = torch.bernoulli(probability_matrix).bool()
        labels[~masked_indices] = -100  # We only compute loss on masked tokens

        # 80% of the time, replace masked input tokens with [MASK]
        indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
        input_ids[indices_replaced] = self.tokenizer.convert_tokens_to_ids(self.tokenizer.mask_token)

        # 10% of the time, replace masked input tokens with random word
        indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
        random_words = torch.randint(len(self.tokenizer), labels.shape, dtype=torch.long)
        input_ids[indices_random] = random_words[indices_random]

        # The rest 10% of the time, keep the masked input tokens unchanged

        return input_ids, labels

if __name__ == '__main__':
    input_dir = '/content/drive/MyDrive/NLP/Dataset/240527/'
    VALID_FILE = input_dir + '[haein]dataset_cleansing+koSpacing+drop_duplicate_240527.csv'

    task = 'test_ctx'  # or 'test_utt'

    if task == 'test_utt':
        dataset = DialogTransformerDataset(VALID_FILE, tokenizer, utt_masklm=True, utt_sop=True)
    elif task == 'test_ctx':
        dataset = DialogTransformerDataset(VALID_FILE, tokenizer, context_shuf=True, context_masklm=False)
    else:
        dataset = DialogTransformerDataset(VALID_FILE, tokenizer)

    train_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=1)
    #print(train_loader.head())
    eval_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=32, shuffle=False, num_workers=1)

    model = BertForMaskedLM.from_pretrained('skt/kobert-base-v1')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    learning_rate = 5e-5
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    criterion = CrossEntropyLoss()

    epochs = 1
    history = {'train_loss': [], 'eval_loss': [], 'train_accuracy': [], 'eval_accuracy': [], 'train_perplexity': [], 'eval_perplexity': []}
    all_predictions = []
    all_targets = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            optimizer.zero_grad()
            context, context_attn_mask, _, _, _, _, _ = batch

            input_ids = context.to(device)
            attention_mask = context_attn_mask.to(device)
            labels = input_ids  # Assuming labels are the same as input_ids for MLM

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = torch.argmax(logits, dim=-1)
            correct_predictions += (preds == labels).sum().item()
            total_predictions += labels.numel()

        avg_train_loss = running_loss / len(train_loader)
        train_accuracy = correct_predictions / total_predictions
        train_perplexity = math.exp(avg_train_loss)

        history['train_loss'].append(avg_train_loss)
        history['train_accuracy'].append(train_accuracy)
        history['train_perplexity'].append(train_perplexity)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Train Perplexity: {train_perplexity:.4f}")

        model.eval()
        eval_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        eval_predictions = []
        eval_targets = []

        with torch.no_grad():
            for batch in tqdm(eval_loader, desc=f"Evaluation Epoch {epoch+1}"):
                context, context_attn_mask, _, _, _, _, _ = batch

                input_ids = context.to(device)
                attention_mask = context_attn_mask.to(device)
                labels = input_ids  # Assuming labels are the same as input_ids for MLM

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                logits = outputs.logits

                eval_loss += loss.item()
                preds = torch.argmax(logits, dim=-1)
                correct_predictions += (preds == labels).sum().item()
                total_predictions += labels.numel()

                eval_predictions.extend(preds.cpu().numpy())
                eval_targets.extend(labels.cpu().numpy())

        avg_eval_loss = eval_loss / len(eval_loader)
        eval_accuracy = correct_predictions / total_predictions
        eval_perplexity = math.exp(avg_eval_loss)

        history['eval_loss'].append(avg_eval_loss)
        history['eval_accuracy'].append(eval_accuracy)
        history['eval_perplexity'].append(eval_perplexity)

        print(f"Epoch {epoch+1}/{epochs}, Eval Loss: {avg_eval_loss:.4f}, Eval Accuracy: {eval_accuracy:.4f}, Eval Perplexity: {eval_perplexity:.4f}")

        all_predictions.extend(eval_predictions)
        all_targets.extend(eval_targets)

    print("Training complete.")
'''

'\nimport pandas as pd\nimport torch\nimport math\nfrom transformers import BertTokenizer, BertForMaskedLM, AdamW\nfrom torch.nn import CrossEntropyLoss\nfrom tqdm import tqdm\n\ndef load_dict(filename):\n    import json\n    with open(filename, "r") as f:\n        return json.loads(f.readline())\n\ndef load_vecs(fin):\n    import tables\n    import numpy as np\n    """read vectors (2D numpy array) from a hdf5 file"""\n    h5f = tables.open_file(fin)\n    h5vecs = h5f.root.vecs\n    vecs = np.zeros(shape=h5vecs.shape, dtype=h5vecs.dtype)\n    vecs[:] = h5vecs[:]\n    h5f.close()\n    return vecs\n\ndef save_vecs(vecs, fout):\n    import tables\n    fvec = tables.open_file(fout, \'w\')\n    atom = tables.Atom.from_dtype(vecs.dtype)\n    filters = tables.Filters(complib=\'blosc\', complevel=5)\n    ds = fvec.create_carray(fvec.root, \'vecs\', atom, vecs.shape, filters=filters)\n    ds[:] = vecs\n    print(\'done\')\n    fvec.close()\n\nclass DialogTransformerDataset(torch.utils.data.Data

In [78]:
#DialogBERT & KoBERT Masking
import pandas as pd
import torch
import math
import random
from transformers import BertTokenizer, BertForMaskedLM, AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

def load_dict(filename):
    import json
    with open(filename, "r") as f:
        return json.loads(f.readline())

def load_vecs(fin):
    import tables
    import numpy as np
    """read vectors (2D numpy array) from a hdf5 file"""
    h5f = tables.open_file(fin)
    h5vecs = h5f.root.vecs
    vecs = np.zeros(shape=h5vecs.shape, dtype=h5vecs.dtype)
    vecs[:] = h5vecs[:]
    h5f.close()
    return vecs

def save_vecs(vecs, fout):
    import tables
    fvec = tables.open_file(fout, 'w')
    atom = tables.Atom.from_dtype(vecs.dtype)
    filters = tables.Filters(complib='blosc', complevel=5)
    ds = fvec.create_carray(fvec.root, 'vecs', atom, vecs.shape, filters=filters)
    ds[:] = vecs
    print('done')
    fvec.close()

class DialogTransformerDataset(torch.utils.data.Dataset):
    def __init__(self, file_path, tokenizer, min_num_utts=2, max_num_utts=15, max_utt_len=50, block_size=512,
                 utt_masklm=False, utt_sop=False, context_shuf=False, context_masklm=False):
        self.tokenizer = tokenizer
        self.min_num_utts = min_num_utts
        self.max_num_utts = max_num_utts
        self.max_utt_len = max_utt_len
        self.block_size = block_size
        self.utt_masklm = utt_masklm
        self.utt_sop = utt_sop
        self.context_shuf = context_shuf
        self.context_masklm = context_masklm

        print("Loading data...")
        df = pd.read_csv(file_path)
        self.contexts = df['corrected_column'].values  # Adjust according to your CSV column names
        # self.knowledge = df['knowledge'].values  # Uncomment if you have this column

        self.masked_indices = set(random.sample(range(len(self.contexts)), len(self.contexts) // 10))  # Mask 10% of the data

    def __len__(self):
        return len(self.contexts)

    def __getitem__(self, idx):
        context = self.contexts[idx]
        encoded_dict = self.tokenizer.encode_plus(
            context,
            add_special_tokens=True,
            max_length=self.max_utt_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_dict['input_ids'].squeeze()
        input_ids_nomask = input_ids.clone()
        attention_mask = encoded_dict['attention_mask'].squeeze()
        target_ids = input_ids.clone()  # Target IDs are the original input IDs

        # Masking
        if idx in self.masked_indices:
            for i in range(len(input_ids)):
                if input_ids[i] not in [
                    self.tokenizer.cls_token_id,
                    self.tokenizer.sep_token_id,
                    self.tokenizer.pad_token_id
                ]:
                    rand = random.random()
                    if rand < 0.8:
                        # 80% chance to replace with [MASK] token
                        input_ids[i] = self.tokenizer.mask_token_id
                    elif rand < 0.9:
                        # 10% chance to replace with a random token
                        input_ids[i] = random.randint(0, self.tokenizer.vocab_size - 1)
                    # Remaining 10% keep the original token
                else:
                    target_ids[i] = -100  # Ignore non-masked tokens during loss computation

        return {
            'input_ids_nomask': input_ids_nomask,
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': target_ids  # Use masked values as labels
        }

if __name__ == '__main__':
    input_dir = '/content/drive/MyDrive/NLP/Dataset/240527/'
    VALID_FILE = input_dir + '[haein]dataset_cleansing+koSpacing+drop_duplicate_240527.csv'

    #tokenizer = BertTokenizer.from_pretrained('skt/kobert-base-v1')
    task = 'test_ctx'  # or 'test_utt'

    if task == 'test_utt':
        dataset = DialogTransformerDataset(VALID_FILE, tokenizer, utt_masklm=True, utt_sop=True)
    elif task == 'test_ctx':
        dataset = DialogTransformerDataset(VALID_FILE, tokenizer, context_shuf=True, context_masklm=False)
    else:
        dataset = DialogTransformerDataset(VALID_FILE, tokenizer)

    train_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=1)
    eval_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=32, shuffle=False, num_workers=1)

    model = BertForMaskedLM.from_pretrained('skt/kobert-base-v1')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    learning_rate = 5e-5
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    criterion = CrossEntropyLoss()

    epochs = 1
    history = {'train_loss': [], 'eval_loss': [], 'train_accuracy': [], 'eval_accuracy': [], 'train_perplexity': [], 'eval_perplexity': []}
    all_predictions = []
    all_targets = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            optimizer.zero_grad()
            input_ids_nomask = batch['input_ids_nomask'].to(device)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)  # Assuming labels are the same as input_ids for MLM

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = torch.argmax(logits, dim=-1)
            correct_predictions += (preds == labels).sum().item()
            total_predictions += labels.numel()

        avg_train_loss = running_loss / len(train_loader)
        train_accuracy = correct_predictions / total_predictions
        train_perplexity = math.exp(avg_train_loss)

        history['train_loss'].append(avg_train_loss)
        history['train_accuracy'].append(train_accuracy)
        history['train_perplexity'].append(train_perplexity)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Train Perplexity: {train_perplexity:.4f}")

        model.eval()
        eval_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        eval_predictions = []
        eval_targets = []

        with torch.no_grad():
            for batch in tqdm(eval_loader, desc=f"Evaluation Epoch {epoch+1}"):
                input_ids_nomask = batch['input_ids_nomask'].to(device)
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)  # Assuming labels are the same as input_ids for MLM

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                logits = outputs.logits

                eval_loss += loss.item()
                preds = torch.argmax(logits, dim=-1)
                correct_predictions += (preds == labels).sum().item()
                total_predictions += labels.numel()

                eval_predictions.extend(preds.cpu().numpy())
                eval_targets.extend(labels.cpu().numpy())

        avg_eval_loss = eval_loss / len(eval_loader)
        eval_accuracy = correct_predictions / total_predictions
        eval_perplexity = math.exp(avg_eval_loss)

        history['eval_loss'].append(avg_eval_loss)
        history['eval_accuracy'].append(eval_accuracy)
        history['eval_perplexity'].append(eval_perplexity)

        print(f"Epoch {epoch+1}/{epochs}, Eval Loss: {avg_eval_loss:.4f}, Eval Accuracy: {eval_accuracy:.4f}, Eval Perplexity: {eval_perplexity:.4f}")

        all_predictions.extend(eval_predictions)
        all_targets.extend(eval_targets)

    print("Training complete.")


Loading data...


Some weights of BertForMaskedLM were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training Epoch 1: 100%|██████████| 1502/1502 [07:09<00:00,  3.50it/s]


Epoch 1/1, Train Loss: 0.6938, Train Accuracy: 0.8623, Train Perplexity: 2.0012


Evaluation Epoch 1: 100%|██████████| 1502/1502 [02:13<00:00, 11.22it/s]

Epoch 1/1, Eval Loss: 0.4727, Eval Accuracy: 0.8913, Eval Perplexity: 1.6044
Training complete.


In [92]:
valid_targets = [[token for token in target if token != -100] for target in all_targets]

In [83]:
# Decoding predictions and targets
decoded_predictions = [tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(pred)) for pred in all_predictions]

In [91]:
print(decoded_predictions[5])

[PAD] 안녕하세요. 저는 요즘 정말 힘들어요.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]


In [93]:
decoded_targets = [tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(target)) for target in valid_targets]

In [95]:

# Display some decoded predictions and targets
for i in range(100):
    print(f"Prediction {i+1}: {decoded_predictions[i]}")
    print(f"Target {i+1}: {decoded_targets[i]}")
    print()

Prediction 1: [PAD] 네, 안녕하세요.요즘 정말 좋은 일이 많이 있었거든요.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]
Target 1: [CLS] 네, 안녕하세요.요즘 정말 좋은 일이 많이 있었거든요.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]

Prediction 2: [PAD] 네, 맞아요.정말 행복한 시기를 보내고 있어요.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]
Target 2: [CLS] 네, 맞아요.정말 행복한 시기를 보내고 있어요.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]

Prediction 3: [PAD] 그렇죠. 어려운 시기를 견뎌내고 이렇게 행복한 날들을 맞이할 수 있어 감사해요.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][

## loss(cross-entropy)

# 첫번째 배치 확인 작업